In [1]:
!git clone https://github.com/IBM-EPBL/IBM-Project-2629-1658478419.git

Cloning into 'IBM-Project-2629-1658478419'...
remote: Enumerating objects: 655, done.
remote: Counting objects: 100% (275/275), done.
remote: Compressing objects: 100% (152/152), done.
remote: Total 655 (delta 116), reused 261 (delta 111), pack-reused 380
Receiving objects: 100% (655/655), 17.76 MiB | 11.31 MiB/s, done.
Resolving deltas: 100% (235/235), done.


In [2]:
import requests

import json
# NOTE: you must manually set API_KEY below using information retrieved from your IBM Cloud account.
API_KEY = "IYhl2VJPcsKa1DvohrhA8BwB8Wk43ShPuQ1VzL0fPMXJ"
token_response = requests.post('https://iam.cloud.ibm.com/identity/token', data={"apikey":
 API_KEY, "grant_type": 'urn:ibm:params:oauth:grant-type:apikey'})
mltoken = token_response.json()["access_token"]

header = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + mltoken}


In [3]:
'''import pickle

filen = "/content/IBM-Project-2629-1658478419/Project Development Phase/Sprint 2/Models/KNN_rainfall.pkl"
model = pickle.load(open(filen, 'rb'))
'''

'import pickle\n\nfilen = "/content/IBM-Project-2629-1658478419/Project Development Phase/Sprint 2/Models/KNN_rainfall.pkl"\nmodel = pickle.load(open(filen, \'rb\'))\n'

In [4]:
!pip install flask-ngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
!pip install pyngrok==4.1.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for pyngrok: filename=pyngrok-4.1.1-py3-none-any.whl size=15982 sha256=db3eb5717e5d2917f9fc86b3429a788417db743bcf2fd19c71a63a4cfeeb0ba7
  Stored in directory: /root/.cache/pip/wheels/b1/d9/12/045a042fee3127dc40ba6f5df2798aa2df38c414bf533ca765
Successfully built pyngrok


In [6]:
!ngrok authtoken 23Jg2oDy2N9PJD3njx9iWFHSfCs_3J7KXYfQXtzxHX3a5digi

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [7]:
import pandas as pd

df = pd.read_csv("/content/IBM-Project-2629-1658478419/Project Development Phase/Sprint 1/weatherAUS.csv")

In [8]:
from flask import Flask, render_template, request
from flask_ngrok import run_with_ngrok
from datetime import datetime
app = Flask(__name__,template_folder="/content/IBM-Project-2629-1658478419/Project Development Phase/Sprint 3/HTML Files/")
run_with_ngrok(app)

@app.route("/")
def prec():
  return render_template("Precautions.html")

@app.route("/pred_in")
def pred_in():
  return render_template("pred_in.html")

@app.route("/pred_out")
def pred_out():
  MinTemp = request.args.get("MinTemp")
  MaxTemp = request.args.get("MaxTemp")
  Rainfall = request.args.get("Rainfall")
  Evaporation = request.args.get("Evaporation")
  Sunshine = request.args.get("Sunshine")
  WindGustSpeed = request.args.get("WindGustSpeed")
  WindSpeed9am = request.args.get("WindSpeed9am")
  WindSpeed3pm = request.args.get("WindSpeed3pm")
  Humidity9am = request.args.get("Humidity9am")
  Humidity3pm = request.args.get("Humidity3pm")
  Pressure9am = request.args.get("Pressure9am")
  Pressure3pm = request.args.get("Pressure3pm")
  Cloud9am = request.args.get("Cloud9am")
  Cloud3pm = request.args.get("Cloud3pm")
  Temp9am = request.args.get("Temp9am")
  Temp3pm = request.args.get("Temp3pm")
  lis = [[MinTemp, MaxTemp, Rainfall, Evaporation, Sunshine, WindGustSpeed, WindSpeed9am, WindSpeed3pm, Humidity9am, Humidity3pm, Pressure9am, Pressure3pm ,Cloud9am, Cloud3pm, Temp9am, Temp3pm	]]
  payload_scoring = {"input_data": [{"field": [["MinTemp", "MaxTemp", "Rainfall", "Evaporation", "Sunshine", "WindGustSpeed", "WindSpeed9am", "WindSpeed3pm", "Humidity9am", "Humidity3pm", 
                                              "Pressure9am", "Pressure3pm" ,"Cloud9am", "Cloud3pm", "Temp9am", "Temp3pm"]], 
                                             "values": lis}]}
  response_scoring = requests.post('https://us-south.ml.cloud.ibm.com/ml/v4/deployments/988a35a4-5d94-4dff-81e8-0fd5e93519f4/predictions?version=2022-11-22', json=payload_scoring,
 headers={'Authorization': 'Bearer ' + mltoken})
  pred = response_scoring.json() 
  p = pred['predictions'][0]['values'][0][0]
  return render_template("pred_out.html", Prediction_result = p)

@app.route("/trend_in")
def trend_in():
  return render_template("trend_in.html")

@app.route("/trend_out")
def trend_out():
  loc = request.args.get("locations")
  yr = request.args.get("year")
  df['Date'] = pd.to_datetime(df['Date'])  
  sd = str(yr)+'-1-1'
  sd = datetime.strptime(sd, '%Y-%m-%d')
  ed = str(yr)+'-12-31'
  ed = datetime.strptime(ed, '%Y-%m-%d')
  date_mask = (df['Date'] >= sd ) & (df['Date'] <= ed)
  loc_mask = (df['Location']==loc)
  mask = date_mask & loc_mask
  data = df.loc[mask]
  return render_template("trend_out.html", tab = data.to_html(), loc=loc, yr = yr)

if __name__ == "__main__":
    app.run() 

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://6103-34-126-138-133.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [22/Nov/2022 17:22:05] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [22/Nov/2022 17:22:06] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [22/Nov/2022 17:22:10] "GET /pred_in HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [22/Nov/2022 17:23:09] "GET /pred_out?Date=2022-11-24&location=NorfolkIsland&MinTemp=12.3&MaxTemp=23.2&Rainfall=12.3&Evaporation=33.2&Sunshine=2.0&WindGustDir=SW&WindGustSpeed=12&WindDir9am=ENE&WindDir3PM=ENE&WindSpeed9am=21&WindSpeed3pm=32&Humidity9am=12&Humidity3pm=23&Pressure9am=23&Pressure3pm=23&Cloud9am=12&Cloud3pm=12&Temp9am=23&Temp3pm=12&RainToday=Yes HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [22/Nov/2022 17:23:11] "GET /trend_in HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [22/Nov/2022 17:23:17] "GET /trend_out?locations=NorfolkIsland&year=2012 HTTP/1.1" 200 -
